# Blood glucose prediction

[Data description](https://www.sciencedirect.com/science/article/pii/S2352340924005262#bib0005)

[Paper](https://www.mdpi.com/1424-8220/21/21/7090)


**Aim of project**: Predict glucose values 1h in advance (with special focus on hyper-/hypoglycemia)

# Setup
Download the data from the published repository and load the preprocessed data in pandas

In [1]:
import os
data_folder = 'AdvancedTimeSeriesCourse-Engineering-Data/Data'
if not os.path.exists(data_folder):
  !git clone https://github.com/ChristopherKunze-Git/AdvancedTimeSeriesCourse-Engineering-Data

Cloning into 'AdvancedTimeSeriesCourse-Engineering-Data'...
remote: Enumerating objects: 193, done.
remote: Counting objects: 100% (110/110), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 193 (delta 53), reused 18 (delta 6), pack-reused 83 (from 1)
Receiving objects: 100% (193/193), 55.45 MiB | 7.89 MiB/s, done.
Resolving deltas: 100% (88/88), done.
Updating files: 100% (31/31), done.


In [2]:
!pip -qq install statsforecast[plotly]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.8/275.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.7/80.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.4/354.4 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 9.3 MB/s eta 0:00:00


In [3]:
from pathlib import Path
import glob
import os
import random

import pandas as pd
import numpy as np

from statsforecast import StatsForecast
import statsforecast.models as sf_models
from utilsforecast.losses import mape, rmse, smape
from utilsforecast.evaluation import evaluate
from utilsforecast.plotting import plot_series

import plotly.graph_objects as go
from scipy.stats import zscore

In [4]:
def add_time_features(df, time_id='time'):
    df['day_of_week'] = df[time_id].dt.dayofweek
    df['month'] = df[time_id].dt.month
    df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
    df['time_of_day'] = df[time_id].dt.hour + df[time_id].dt.minute / 60
    df['hour_sin'] = np.sin(2 * np.pi * df['time_of_day'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['time_of_day'] / 24)
    return df

def calc_time_since_event(df, event_mask, freq=1, cap=np.inf):
    not_event = ~event_mask
    time_since_event = (not_event.groupby((not_event != not_event.shift()).cumsum())
                              .transform('cumsum'))
    time_since_event = time_since_event / freq
    time_since_event = time_since_event.where(time_since_event <= cap, cap)
    return time_since_event


def show_interactive_ts_plot(df, var_names = None, normalization="None", title="",
                             show_first_only=False, groups={}, layout=None):
    """
    input:
        df: pd.DataFrame
        var_names: list of column names
        normalization: 'None', 'MinMax', 'ZScore' - which normalization to apply
        hide_all: bool - if True, all traces will be hidden at first
    """
    if var_names is None:
        var_names = df.select_dtypes(include=['number']).columns.to_list()

    for k, col in enumerate(var_names):
        if normalization == 'MinMax':
            df[col] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())
        elif normalization == 'ZScore':
            df[col] = zscore(df[col])

    fig = go.Figure()
    for k, col in enumerate(var_names):
        if groups:
            group = next(key for key, val in groups.items() if col in val)
            first_group = next(iter(groups))
            visible = 'legendonly' if show_first_only and group!=first_group else True
            fig.add_trace(go.Scatter(x=df.index, y=df[col], name=col, visible=visible,
                                     legendgroup=group, legendgrouptitle_text=group))
        else:
            visible = 'legendonly' if show_first_only and (k>0) else True
            fig.add_trace(go.Scatter(x=df.index, y=df[col], name=col, visible=visible))
    fig.update_layout(
        title = title,
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=1,
                        label="1d",
                        step="day",
                        stepmode="backward"),
                    dict(count=7,
                        label="1w",
                        step="day",
                        stepmode="backward"),
                    dict(count=1,
                        label="1m",
                        step="month",
                        stepmode="backward"),
                    dict(step="all")
                ])
            ),
            rangeslider=dict(
                visible=True
            ),
            type="date"
        )
    )
    if layout is not None:
        fig.update_layout(layout)

    fig.show()

In [5]:
dfs = {}
for filename in Path(f'{data_folder}').glob('*.csv'):
    print(filename.stem)
    df = pd.read_csv(filename)
    df['time'] = pd.to_datetime(df['time'])
    if 'old_time' in df.columns:
        df['old_time'] = pd.to_datetime(df['old_time'])
    dfs[filename.stem] = df

dfs['train_p27_no_gaps'].head(1)

train_p27_no_gaps
test_multi_no_gaps
test_p27_no_gaps
test_multi
train_multi
test_p27
train_p27
train_multi_no_gaps


,time,old_time,subj,glucose,burned_calories,heart_rate,basal_insulin,bolus_insulin,hour_of_day,hour_sin,hour_cos,heart_rate_imp,glucose_imp
0,2020-06-26 22:15:00,2020-06-26 22:15:00,P27,112.0,22.655359,81.0,0.066,0.0,22,-0.5,0.866025,81.0,112.0


In [6]:
dfs['train_multi_no_gaps_wp27'] = pd.concat([dfs['train_p27_no_gaps'], dfs['train_multi_no_gaps']])
dfs['test_multi_no_gaps_wp27'] = pd.concat([dfs['test_p27_no_gaps'], dfs['test_multi_no_gaps']])

# Baseline model: Naive

A naive / seasonal-naive model serves as a baseline for prediction. Since the original glucose data is measured at 15min interval, not 5min, we could resample the data back to 15min frequency before modeling

In [35]:
freq = '5min' # change to '15min' to first resample data
one_hour = pd.Timedelta('1h') // pd.Timedelta(freq)
one_day = pd.Timedelta('1d') // pd.Timedelta(freq)
one_week = 7*one_day

covariates = []

def get_df(name, covariates=covariates):
    df = dfs[name].copy()
    add_time_features(df, time_id='old_time')
    df['time_since_meal'] = calc_time_since_event(df, df['bolus_insulin'] > 0,
                                                  freq=one_hour, cap=24)
    rename_map = {'subj':'unique_id', 'time': 'ds', 'glucose_imp': 'y'}
    df = (df[['subj', 'time', 'glucose_imp', *covariates]]
              .groupby([pd.Grouper(key='time', freq=freq), 'subj']).first().reset_index()
              .rename(columns=rename_map))
    return df


def eval_task(train_key, test_key):
    # prepare train and test df
    df_train = get_df(train_key)
    df_test = get_df(test_key)

    # setup models
    models=[sf_models.Naive(),
            sf_models.SeasonalNaive(season_length=one_day),]
    model_names = [model.alias for model in models]

    sf = StatsForecast(
        models,
        freq=freq,
    )

    train_subjs = df_train.unique_id.unique()
    test_subjs = df_test.unique_id.unique()
    print(f'Train subjects: {train_subjs}')
    print(f'Test subjects: {test_subjs}')

    # the model will only use data of the same subject max 1 day ago
    X = df_train[df_train.unique_id.isin(test_subjs)].iloc[-7*one_day:]
    y = df_test

    # train the model
    if len(X) > one_day:
        res_df = sf.fit(X)

    preds = []
    for subj in test_subjs:
        X_s = X.loc[X.unique_id == subj]
        y_s = y.loc[y.unique_id == subj]

        cv_df = sf.cross_validation(
            df = y_s,#pd.concat([X_s, y_s]),
            h = one_hour,
            step_size = 1,
            test_size = len(y_s)-one_hour,
            n_windows = None,
          )
        # we just need the 1h-point forecast
        cv_df.drop(
            columns='cutoff',
            index = cv_df[cv_df.ds.duplicated(keep='first')].index,
            inplace=True)
        preds.append(cv_df)
    preds = pd.concat(preds)

    scores = evaluate(preds, metrics=[mape, rmse], agg_fn='mean').set_index('metric')

    return preds, scores

In [36]:
preds, scores = eval_task('train_p27_no_gaps', 'test_p27_no_gaps')
show_interactive_ts_plot(preds.set_index('ds'), ['y', 'Naive', 'SeasonalNaive'])
scores.loc['mape'] = 100*scores.loc['mape']
display(scores)

Train subjects: ['P27']
Test subjects: ['P27']


,Naive,SeasonalNaive
metric,,
mape,16.779164,42.424679
rmse,28.070241,62.917401


## Eval all tasks

In [37]:
tasks = [
    ('Task 1a', 'train_p27_no_gaps', 'test_p27_no_gaps'),
    ('Task 1b', 'train_multi_no_gaps_wp27', 'test_p27_no_gaps'),
    ('Task 2a', 'train_p27_no_gaps', 'test_multi_no_gaps'),
    ('Task 2b', 'train_multi_no_gaps_wp27', 'test_multi_no_gaps')
]

for task, train_key, test_key in tasks:
    print(f'{task}: {train_key} -> {test_key}')
    preds, scores = eval_task(train_key, test_key)
    scores.loc['mape'] = 100*scores.loc['mape']
    #show_interactive_ts_plot(preds.set_index('ds'), ['y', 'Naive', 'SeasonalNaive'])
    display(scores.round(2))

Task 1a: train_p27_no_gaps -> test_p27_no_gaps
Train subjects: ['P27']
Test subjects: ['P27']


,Naive,SeasonalNaive
metric,,
mape,16.78,42.42
rmse,28.07,62.92


Task 1b: train_multi_no_gaps_wp27 -> test_p27_no_gaps
Train subjects: ['P01' 'P03' 'P02' 'P05' 'P04' 'P14' 'P16' 'P17' 'P19' 'P20' 'P25' 'P23'
 'P22' 'P24' 'P26' 'P27' 'P28']
Test subjects: ['P27']


,Naive,SeasonalNaive
metric,,
mape,16.78,42.42
rmse,28.07,62.92


Task 2a: train_p27_no_gaps -> test_multi_no_gaps
Train subjects: ['P27']
Test subjects: ['P06' 'P07' 'P11' 'P15' 'P18' 'P21']


,Naive,SeasonalNaive
metric,,
mape,24.77,51.05
rmse,47.45,85.92


Task 2b: train_multi_no_gaps_wp27 -> test_multi_no_gaps
Train subjects: ['P01' 'P03' 'P02' 'P05' 'P04' 'P14' 'P16' 'P17' 'P19' 'P20' 'P25' 'P23'
 'P22' 'P24' 'P26' 'P27' 'P28']
Test subjects: ['P06' 'P07' 'P11' 'P15' 'P18' 'P21']


,Naive,SeasonalNaive
metric,,
mape,24.77,51.05
rmse,47.45,85.92
